<a href="https://colab.research.google.com/github/jeffreyyao128/qoptimize/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.integrate import solve_ivp
import jax
from jax import jit,vmap,grad
from jax import random
import numpy as np
import jax.numpy as jnp
from jax import vjp


N = 4 # set number of qubits as a global parameter
N1 = 10 #fourier series
key = random.PRNGKey(42)

In [ ]:
def generate_bach(U,N=100,*args):
    '''
    To generate a bach of data point
    '''
    pass

To perceed neural network, an ODE solver is needed. In our case is `scipy.integrate.solve_ivp`

In [ ]:
def forward(t1,psi0,flat_p):
    '''
    solver for Schrodinger equation with initial condition of psi0 and Hamiltonian in the interval of [0,t1]
    use mathod of scipy
    '''
    Hz = lambda t,x: H(t,x,flat_p,t1)
    sol = solve_ivp(Hz,[0,t1],psi0,t_eval=[0,t1])
    return sol.t,sol.y

In [ ]:
def initial():
    '''
    Return randomized parameters
    '''
    return np.random.normal(size=(N1*(N+1)+1,))

In [ ]:
def n(i,x):
    '''
    number operator for n_i (x)
    '''
    return (x&2**i)/2**i

def unpackp(x):
  return jnp.array(x[:N1]) , jnp.array([x[(i+1)*N1:(i+2)*N1] for i in range(N)]), x[-1]

def flatten(x):
  return 0 # TODO

Our Hamiltonian is $$\begin{aligned}
\frac{H}{\hbar}=& \frac{\Omega(t)}{2} \sum_{i=1}^{N} \sigma_{x}^{(i)}-\sum_{i=1}^{N} \Delta_{i}(t) n_{i} \\
&+\sum_{i<j} \frac{V}{|i-j|^{6}} n_{i} n_{j}
\end{aligned}$$

In [ ]:
def H(t,psi,flat_p,t1):
    '''
    Using xor to implement off diagonal element
    '''
    D, = jnp.shape(psi)
    res = jnp.zeros(D,dtype=jnp.complex64)

    w = 2*jnp.pi/t1

    u_omega, u_d, V = unpackp(flat_p)
    omega = jnp.sum(jnp.array([u_omega[i]*jnp.sin(w*(i+1)*t) for i in range(N1)]))
    delta = [jnp.sum(jnp.array([u_d[j,i]*jnp.sin(w*(i+1)*t) for i in range(N1)])) for j in range(N)]

    for x in range(D):
        Ci = psi[x]
        diag = - jnp.sum(jnp.array([ delta[i]*n(i,x) for i in range(N)]))\
        + jnp.sum(jnp.array([[V*n(i,x)*n(j,x)/jnp.abs(i-j)**6 if i<j else 0 for i in range(N)] for j in range(N)])) #diagonal part of hamiltonian
        res = jax.ops.index_add(res, (x,), diag*Ci)
        cast = jnp.array([x^2**i for i in range(N)]) # ^ is for xor, to calculate the flip operation
        res = jax.ops.index_add(res, cast, omega*Ci/2)
    return res

In [ ]:
@jit
def loss(psi,p,t1,A,psi0):
    '''
    define the loss function
    '''
    omega = 2*jnp.pi/t1
    l1 = 1 - jnp.abs(jnp.dot(jnp.conjugate(psi),psi0))**2 # Overlap
    
    l2 = 0.5*jnp.sum(jnp.array([ (i*omega*p[i])**2 for i in range(N1)]))
    l3 = 0.5*jnp.sum(jnp.array([ (i*omega*p[i+N1])**2 for i in range(N1)]))
    l4 = 0.5*p[-1]**2
    
    return A[0]*l1+A[1]*l2+A[2]*l3+A[3]*l4

In [ ]:
@jit
def grad_all(psi1,flat_p,t1,A,psi0):
    
    D, = jnp.shape(psi1)
    
    a0 = grad(loss, argnums=0)(psi1,flat_p,t1,A,psi0)
    pLt1 = jnp.array(jnp.dot(a0,H(t1,psi1,flat_p,t1)))
    print(pLt1)
    aug_state = jnp.concatenate([psi1, a0, -pLt1, jnp.zeros((2*N1+1,))])

    def unpack(x):
              # z , vjp_z , vjp_t , vjp_args
      return x[:D], x[D:2*D], x[2*D], x[2*D+1:]
    def wrap_vjp(augment_state,t,parameters):
      Hz = lambda z : H(t,z,parameters,t1)
      Ht = lambda t0 : H(t0,augment_state[0],parameters,t1)
      Hp = lambda p : H(t,auement_state[0],p,t1)

      _, vjp_funz = vjp(Hz, augment_state[0])
      _, vjp_funt = vjp(Ht, t)
      _, vjp_funp = vjp(Hp, flat_p)

      return [- vjp_funz(augment_state[1]),- vjp_funp(augment_state[1]),- vjp_funt(augment_state[1])]
    
    def aug_dynamics(augment_state,t,parameters):
      unpacked_aug_state = unpack(augment_state)
      vjps = wrap_vjp(unpacked_aug_state,t,parameters)
      return jnp.concatenate([H(t,augment_state[0],parameters,t1), vjps[0], vjps[1], vjps[2]])

    aug_state0 = solve_ivp(aug_dynamics,[t1,0],aug_state)
    return aug_state0

In [ ]:
if __name__=='__main__':
  psi0 = np.random.normal(size=(16,)) + 1j*np.random.normal(size=(16,))
  flat_params = initial()
  res = forward(2,psi0,flat_params)

In [ ]:
z1 = jnp.array(res[1][:,1])
A = np.random.uniform(size=(4,))
grad_all(z1,flat_params,2,A,psi0)

Traced<ShapedArray(complex64[])>with<DynamicJaxprTrace(level=0/1)>


TypeError: ignored